# DATA COLLECTION

Data collection script - for road following

Scripts are based and tested on image, "Yahboom_jetbot_64G_20200520"

Original source code from https://github.com/NVIDIA-AI-IOT/jetbot/

Modified for P-project, needs road following with traffic sign detection.


### 1. Import Libraries

Import libraries to start data collection

In [ ]:
import ipywidgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display

from jetbot import Robot, Camera, bgr8_to_jpeg

from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

### 2. Collect data
Collect data with clickable widget


Create directory for road following : ``dataset_xy``

When live camera feed starts, Click the path on the image.

Left side -> Live feed // Right side -> Saved image

To uniquely identifies the file, use ``uuid1`` method.

In [ ]:
from jupyter_clickable_image_widget import ClickableImageWidget

DATASET_DIR = 'dataset_xy'

try:
    os.makedirs(DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')

camera = Camera()

camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)

count_widget = ipywidgets.IntText(description='count')
count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1())
        image_path = os.path.join(DATASET_DIR, uuid + '.jpg')
        with open(image_path, 'wb') as f:
            f.write(camera_widget.value)
        
        snapshot = camera.value.copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, snapshot_widget]),
    count_widget
])

display(data_collection_widget)

### 3. Close and Send dataset
Close the camera, compess dataset folder to copy easier

After running this code, the file ``dataset_xy.zip`` will be created

Find this file in the Jupyter Lab file browser, right click and select ``copy download link``

Using the link, download the file to computer.

In [ ]:
camera.stop()

!zip -r -q dataset_xy.zip dataset_xy